In [118]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
import put101 as p
import dotenv
import os
import MetaTrader5 as mt5
from MetaTrader5 import AccountInfo, TerminalInfo
from datetime import datetime
import data_utils
import logging

import importlib
importlib.reload(mt5)
importlib.reload(p)



<module 'put101' from 'C:\\Users\\puche\\code\\nautilus\\put101\\__init__.py'>

# 1. Connect to MT5

In [120]:

# load data from the MetaTrader 5 terminal into parquet files for further backtesting
if not dotenv.load_dotenv():
    logging.log(logging.INFO, "No .env file found")

MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.environ["CATALOG_PATH"]

loader_config = p.MTLoginConfig(server=MT5_SERVER, login=MT5_LOGIN, password=MT5_PASSWORD)

venue_EIGHTCAP = "SIM_EIGHTCAP"
loader = p.MT5Loader(data_path=DATA_PATH, catalog_path=CATALOG_PATH, config=loader_config, venue=venue_EIGHTCAP)



In [121]:
loader.init()

True

# Symbol Parameters

In [126]:
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.data import BarType, BarSpecification, BarAggregation
# load a couple of symbols into the catalog using the loader
symbol_broker = 'EURUSD.i'
symbol_clean = 'EURUSD'
timeframe = mt5.TIMEFRAME_M1
start_date = datetime(1971, 1, 1)
end_date = datetime(2023,12,2)


# variables dependent on parameters
if symbol_clean:
    symbol = symbol_clean
else:
    symbol = symbol_broker

# Delete the symbol from the catalog

In [125]:
# delete the symbols data from the catalog
bar_type = loader.get_bar_type(symbol, timeframe)
importlib.reload(data_utils)
print(bar_type)

if not data_utils.delete_parquet_data(bar_type, CATALOG_PATH):
    print(f"INFO: Could not delete")
else:
    print(f"INFO: Deleted data for {bar_type}")
    
    

EURUSD.SIM_EIGHTCAP-1-MINUTE-LAST-EXTERNAL
[WinError 3] Das System kann den angegebenen Pfad nicht finden: 'catalog\\data\\bar\\EURUSD.SIM_EIGHTCAP-1-MINUTE-LAST-EXTERNAL'
INFO: Could not delete


# Load the symbol from mt5 to csv

In [127]:
loader.load_symbol_rates_to_csv(symbol_broker, symbol_clean, timeframe, start_date, end_date, DATA_PATH)

INFO: Saved rates to csv: data\EURUSD.i_1-MINUTE_31532400_1701471600.csv


True

# Load the symbol from csv to parquet catalog

In [128]:
loader.load_csv_to_catalog(symbol_broker, symbol_clean, timeframe, start_date, end_date)

True

In [75]:
instrument = loader.get_instrument_FOREX(symbol=symbol)

INFO: Loaded EURUSD.SIM with bartype 16385-MINUTE-MID into catalog
